In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('ucsd-sansdescrip03162018.csv', sep="|")

In [3]:
styles_df = df.drop(axis=1, labels=['Entered','RateCount','BrewerCity','Abbrev','BrewerZIPCode'])

styles_df.replace(to_replace=["Altbier","Amber Ale","American Pale Ale","American Strong Ale","American Strong Ale ","Barley Wine","Bitter","Brown Ale","Cream Ale","English Pale Ale","English Strong Ale","Golden Ale/Blond Ale","Imperial IPA","India Pale Ale (IPA)","India Pale Ale &#40;IPA&#41;","Irish Ale","Kölsch","Mild Ale","Old Ale","Premium Bitter/ESB","Scotch Ale","Scottish Ale","Session IPA"], value="Anglo-American Ales",inplace=True)
styles_df.replace(to_replace=["Amber Lager/Vienna","California Common","Czech Pilsner (Světlý)","Czech Pilsner (Sv&#283;tlý)","Doppelbock","Dortmunder/Helles","Dunkel/Tmavý","Dunkler Bock","Eisbock","Heller Bock","Imperial Pils/Strong Pale Lager","India Style Lager","Malt Liquor","Oktoberfest/Märzen","Pale Lager","Pilsener","Polotmavý","Premium Lager","Radler/Shandy","Schwarzbier","Zwickel/Keller/Landbier"], value="Lagers",inplace=True)
styles_df.replace(to_replace=["Abbey Dubbel","Abbey Tripel","Abt/Quadrupel","Belgian Ale","Belgian Strong Ale","Bière de Garde","Saison"], value="Belgian-Style Ales",inplace=True)
styles_df.replace(to_replace=["Baltic Porter","Black IPA","Dry Stout","Foreign Stout","Imperial Porter","Imperial Stout","Porter","Stout","Sweet Stout"], value="Stout and Porter",inplace=True)
styles_df.replace(to_replace=["Dunkelweizen","German Hefeweizen","German Kristallweizen","Grodziskie/Gose/Lichtenhainer","Weizenbock","Wheat Ale","Witbier"], value="Wheat Beer",inplace=True)
styles_df.replace(to_replace=["Berliner Weisse","Lambic Style - Faro","Lambic Style - Fruit","Lambic Style - Gueuze","Lambic Style - Unblended","Sour Red/Brown","Sour/Wild Ale"], value="Sour Beer",inplace=True)
styles_df.replace(to_replace=["Fruit Beer","Low Alcohol","Sahti/Gotlandsdricke/Koduõlu","Smoked","Specialty Grain","Spice/Herb/Vegetable","Traditional Ale"], value="Other",inplace=True)
styles_df.replace(to_replace=["Cider","Ice Cider/Ice Perry","Mead","Perry","Saké - Daiginjo","Saké - Futsu-shu","Saké - Genshu","Saké - Ginjo","Saké - Honjozo","Saké - Infused","Saké - Junmai","Saké - Koshu","Saké - Namasaké","Saké - Nigori","Saké - Taru","Saké - Tokubetsu"], value="Cider, Mead, Sake",inplace=True)

In [4]:
df['BeerStyleGroup'] = styles_df.BeerStyleName

df.BeerStyleName.replace(to_replace="India Pale Ale &#40;IPA&#41;",value="India Pale Ale", inplace=True)
df.BeerStyleName.replace(to_replace="Czech Pilsner (Sv&#283;tlý)",value="Czech Pilsner (Světlý)", inplace=True)

df = df.drop([285995])

df.to_csv('beers_grouped.csv')


df.BeerStyleName.value_counts()

India Pale Ale                     33411
American Pale Ale                  16908
Imperial IPA                       14052
Saison                             13546
Sour/Wild Ale                      11981
Imperial Stout                     11800
Porter                              9883
Stout                               8603
Spice/Herb/Vegetable                8368
Brown Ale                           8088
Fruit Beer                          7700
Amber Ale                           7206
Golden Ale/Blond Ale                5903
Belgian Ale                         5485
Sweet Stout                         5286
Wheat Ale                           5274
American Strong Ale                 5103
Cider                               4858
Barley Wine                         4360
Belgian Strong Ale                  4216
Berliner Weisse                     3870
Witbier                             3731
Pilsener                            3718
Black IPA                           3694
Mead            

### Nationally, IPA's and Pale Ale's in general overwhelm other styles by production. Let's see if any particular states are the culpirits for this pale ale craze.

In [5]:
states = df.Abbrev.value_counts().index

states_df = pd.DataFrame.from_dict(states)

states_df.columns = ['states']
states_df.sort_values(by=['states'],inplace=True)
states_df.reset_index(drop=True)

def state_style(state):

    return df.loc[df['Abbrev'] == state].BeerStyleName.value_counts().index[:10].values

In [6]:
mostCommonByState = pd.Series()

for row in states_df.itertuples():
    mostCommonStyles10 = state_style(row.states)
    mostCommonByState[row.states] = mostCommonStyles10[0]

In [7]:
mostCommonByState.value_counts()

India Pale Ale       48
American Pale Ale     1
Saison                1
Imperial Stout        1
dtype: int64

### With the exception of one state, the hype around IPA's has affected the entire nation and is only beat out in 3 states, 2 of which still just have a different kind of pale ale at the top. Let's see which beers would be most popular if we ignored pale ales.

In [8]:
for row in states_df.itertuples():
    mostCommonStyles10 = state_style(row.states)
    mostCommonStyles3 = []
    i = 0
    while (len(mostCommonStyles3) < 3):
        if (mostCommonStyles10[i] not in ['India Pale Ale', 'American Pale Ale', 'Imperial IPA', 'Saison']):
            mostCommonStyles3.append(mostCommonStyles10[i])
        i += 1;
    states_df.set_value(col='most_common', index=row.Index, value=mostCommonStyles3[0])
    states_df.set_value(col='second_most', index=row.Index, value=mostCommonStyles3[1])
    states_df.set_value(col='third_most', index=row.Index, value=mostCommonStyles3[2])

In [9]:
states_df.most_common.value_counts()

Sour/Wild Ale           13
Imperial Stout          12
Porter                   8
Mead                     4
Spice/Herb/Vegetable     3
Stout                    3
Amber Ale                3
Fruit Beer               2
Golden Ale/Blond Ale     1
Cider                    1
Barley Wine              1
Name: most_common, dtype: int64

### This ranking looks quite similar to the results of the national style ranking (excluding pale ales), suggesting that there may not be too much variation in style production by region.